In [ ]:
!pip install pyTigerGraph
import requests
import json
import pyTigerGraph as tg

In [ ]:
URL = "http://myvariant.info/v1/variant/query?q=_exists_:dbsnp&fields=dbsnp.rsid,dbsnp.vartype" # URL
res = requests.get(url = URL).json() # Makes the GET Request
print(json.dumps(res, indent = 2)) # Printing the Results

In [ ]:
conn = tg.TigerGraphConnection(host="https://work23.i.tgcloud.io", password="tigergraph", gsqlVersion="3.0.5", useCert=True)

In [ ]:
# Sets up connection with GSQL
print(conn.gsql('ls', options=[]))
 # Create Edges (POKEMON_TYPE) and Vertices (Pokemon and Type)
print(conn.gsql('''CREATE VERTEX Cancers (PRIMARY_ID name STRING) WITH primary_id_as_attribute="true"
                    CREATE VERTEX Variants (PRIMARY_ID type STRING) WITH primary_id_as_attribute="true"
                    CREATE UNDIRECTED EDGE VARIANT_TYPES (FROM Cancers, TO Variants)''', options=[]))
print(conn.gsql('''CREATE GRAPH CancerGraphs(Cancers, Variants, VARIANT_TYPES)''', options=[])) # Create the Graph

In [ ]:
conn.graphname = "CancerGraphs"
conn.apiToken = conn.getToken(conn.createSecret())

In [ ]:
for i in res["hits"]:
  conn.upsertVertex("Cancers", i["_id"], attributes={"name": i["_id"] })

In [ ]:
for ty in res["hits"]:
  conn.upsertVertex("Variants", ty["dbsnp"]["rsid"], attributes={"type": ty["dbsnp"]["rsid"] })
  conn.upsertEdge("Cancers", ty["dbsnp"]["vartype"], "VARIANT_TYPES", "Variants", ty["dbsnp"]["vartype"])